### 2-2. 하나의 네이버 웹툰과 1개의 회차에 대한 Image 다운로드 하기 (필수)
- 하나의 웹툰의 **제목(title)**, **회차 번호(no)**, **회차 URL(url)**를 입력으로 받는 함수를 선언.
- 함수 호출 후, `img/일렉시드/341` 디렉토리가 생성되며, 해당 디렉토리 아래에 웹툰 이미지들이 다운로드되도록 구현한다.

In [ ]:
import os
import re
import requests

In [ ]:
def download_one_episode(title, no, url):
    # 1. 디렉토리 생성
    path = os.path.join("img", title, str(no))
    os.makedirs(path, exist_ok=True)
    print(f"[{path}] 디렉토리 준비 완료.")

    # 2. 헤더 설정 (중요: Referer와 User-Agent가 정확해야 차단 안 됨)
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36',
        'Referer': url
    }

    try:
        # 3. 페이지 소스 가져오기
        response = requests.get(url, headers=headers)
        html_text = response.text

        # 4. 정규표현식으로 이미지 URL 패턴 추출
        # 네이버 웹툰 이미지 서버 주소인 'image-comic.pstatic.net'이 포함된 모든 주소를 찾습니다.
        pattern = r'(https://image-comic\.pstatic\.net/webtoon/[^\s"\']+)'
        img_list = re.findall(pattern, html_text)

        # 중복 제거 및 유효성 검사
        img_list = sorted(list(set(img_list)))
        
        # 실제 본문 이미지만 필터링 (불필요한 썸네일 등 제외)
        final_list = [img for img in img_list if '/IMAG' in img or 'title_img' not in img.lower()]

        if not final_list:
            print("이미지 URL 추출에 실패했습니다. 패턴을 다시 확인하세요.")
            return

        print(f"추출 완료: 총 {len(final_list)}개의 이미지를 다운로드합니다.")

        # 5. 이미지 다운로드 진행
        download_count = 0
        for idx, img_url in enumerate(final_list):
            # 파일명 설정 (001.jpg, 002.jpg...)
            filename = f"{idx + 1:03d}.jpg"
            save_path = os.path.join(path, filename)

            # 이미지 요청
            img_res = requests.get(img_url, headers=headers)
            
            if img_res.status_code == 200:
                with open(save_path, 'wb') as f:
                    f.write(img_res.content)
                print(f"다운로드 완료: {filename}")
                download_count += 1
            else:
                print(f"다운로드 실패: {filename} (HTTP {img_res.status_code})")

        print(f"\n최종 완료: 총 {download_count}개의 이미지가 저장되었습니다.")

    except Exception as e:
        print(f"오류 발생: {e}")

In [ ]:
download_one_episode('일렉시드', 341, 'https://comic.naver.com/webtoon/detail?titleId=717481&no=341&week=wed')

[img\일렉시드\341] 디렉토리 준비 완료.
추출 완료: 총 473개의 이미지를 다운로드합니다.
다운로드 완료: 001.jpg
다운로드 완료: 002.jpg
다운로드 완료: 003.jpg
다운로드 완료: 004.jpg
다운로드 완료: 005.jpg
다운로드 완료: 006.jpg
다운로드 완료: 007.jpg
다운로드 완료: 008.jpg
다운로드 완료: 009.jpg
다운로드 완료: 010.jpg
다운로드 완료: 011.jpg
다운로드 완료: 012.jpg
다운로드 완료: 013.jpg
다운로드 완료: 014.jpg
다운로드 완료: 015.jpg
다운로드 완료: 016.jpg
다운로드 완료: 017.jpg
다운로드 완료: 018.jpg
다운로드 완료: 019.jpg
다운로드 완료: 020.jpg
다운로드 완료: 021.jpg
다운로드 완료: 022.jpg
다운로드 완료: 023.jpg
다운로드 완료: 024.jpg
다운로드 완료: 025.jpg
다운로드 완료: 026.jpg
다운로드 완료: 027.jpg
다운로드 완료: 028.jpg
다운로드 완료: 029.jpg
다운로드 완료: 030.jpg
다운로드 완료: 031.jpg
다운로드 완료: 032.jpg
다운로드 완료: 033.jpg
다운로드 완료: 034.jpg
다운로드 완료: 035.jpg
다운로드 완료: 036.jpg
다운로드 완료: 037.jpg
다운로드 완료: 038.jpg
다운로드 완료: 039.jpg
다운로드 완료: 040.jpg
다운로드 완료: 041.jpg
다운로드 완료: 042.jpg
다운로드 완료: 043.jpg
다운로드 완료: 044.jpg
다운로드 완료: 045.jpg
다운로드 완료: 046.jpg
다운로드 완료: 047.jpg
다운로드 완료: 048.jpg
다운로드 완료: 049.jpg
다운로드 완료: 050.jpg
다운로드 완료: 051.jpg
다운로드 완료: 052.jpg
다운로드 완료: 053.jpg
다운로드 완료: 054.jpg
다운로드 완료: 055.jpg
다운로드 완료: 

### 2-3. 하나의 네이버 웹툰과 여러개의 회차에 대한 Image 다운로드 하기 (선택)
- 하나의 웹툰의 제목(title)과 회차를 알 수 있는 url을 입력으로 받는 함수를 선언
   `def download_all_episode(title, episode_url):`
- 하나의 웹툰에 대한 1Page의 20 회차의 image를 다운로드 받음

In [19]:
!pip install webdriver-manager

In [37]:
import os
import re
import requests
import time
from concurrent.futures import ThreadPoolExecutor # 비동기 처리를 위한 모듈

In [38]:
# 1. 이미지 한 장을 저장하는 함수
def save_image(args):
    img_url, save_path, headers = args
    try:
        res = requests.get(img_url, headers=headers, timeout=10)
        if res.status_code == 200:
            with open(save_path, 'wb') as f:
                f.write(res.content)
            return True
    except:
        return False
    return False

In [39]:
# 2. 한 회차 전체를 처리하는 함수
def download_one_episode(title, no, url):
    path = os.path.join("img", title, str(no))
    os.makedirs(path, exist_ok=True)

    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36',
        'Referer': url
    }

    try:
        response = requests.get(url, headers=headers)
        pattern = r'https://image-comic\.pstatic\.net/webtoon/[^"\'\s>]+'
        img_list = re.findall(pattern, response.text)
        final_list = sorted(list(set([img for img in img_list if '/IMAG' in img])))

        if not final_list:
            return f"{no}화 이미지 없음"

        # 각 이미지 다운로드 작업을 생성
        task_args = []
        for idx, img_url in enumerate(final_list):
            save_path = os.path.join(path, f"{idx + 1:03d}.jpg")
            task_args.append((img_url, save_path, headers))

        # 내부적으로 이미지를 병렬 다운로드 (필요에 따라 조절)
        with ThreadPoolExecutor(max_workers=5) as img_executor:
            img_executor.map(save_image, task_args)

        print(f"   ㄴ [성공] {no}화 다운로드 완료")
        return f"{no}화 완료"
    except Exception as e:
        return f"{no}화 에러: {e}"


In [40]:
# 3. 모든 회차를 비동기로 관리하는 메인 함수
def download_all_episode(title, list_url):
    match = re.search(r'titleId=(\d+)', list_url)
    if not match: return
    title_id = match.group(1)

    api_url = f"https://comic.naver.com/api/article/list?titleId={title_id}&page=1"
    headers = {'User-Agent': 'Mozilla/5.0'}
    
    res = requests.get(api_url, headers=headers)
    articles = res.json().get('articleList', [])

    print(f"[{title}] 총 {len(articles)}개 회차를 동시에 비동기로 다운로드합니다.")

    # 전체 회차에 대한 작업을 리스트로 만듭니다.
    episode_tasks = []
    for article in articles:
        no = article['no']
        detail_url = f"https://comic.naver.com/webtoon/detail?titleId={title_id}&no={no}"
        episode_tasks.append((title, no, detail_url))

    # [핵심] 여러 회차를 동시에 실행 (max_workers=4 정도로 설정하여 서버 차단 방지)
    # 4개의 회차가 동시에 다운로드됩니다.
    with ThreadPoolExecutor(max_workers=4) as ep_executor:
        ep_executor.map(lambda p: download_one_episode(*p), episode_tasks)

    print("\n[모든 작업 완료]")

In [ ]:
download_all_episode('배달왕', 'https://comic.naver.com/webtoon/list?titleId=823933')


[배달왕] 총 20개 회차를 동시에 비동기로 다운로드합니다.
